# **1. Extract information about Justin Trudeau for sysdate-1**

In [1]:
import pandas as pd
import requests
import datetime
import urllib.parse
from datetime import datetime
import pyodbc

# **5. Create a log file to check if any issue happened during the process**

In [2]:
import logging

logging.basicConfig(filename=r"C:\Users\Lion\Desktop\scraper.log",level=logging.DEBUG, format = '%(asctime)s - %(message)s',datefmt='%d-%b-%y %H:%M:%S')

In [3]:
logging.info("Extracting information about Justin Trudeau for sysdate-1")

# define parameters for our request to guardian api for sysdate-1
parameters={
    'from-date' : '2016-01-01',
    'to-date' : str(datetime.today())[:10],
    'q' : 'Justin%20AND%20Trudeau%20OR%20JustinTrudeau',
    'format' : 'json',
    'page' : 1,
    'page-size' : 10,
    'api-key' : "your api key"
            }

# add parameters to the main link 
main_url='https://content.guardianapis.com/search?'
url=main_url+ urllib.parse.urlencode(parameters).replace('%2520', '%20')
data= requests.get(url).json()

In [4]:
# extract result of our responses and add them to a list to make a dataframe later
my_data = []
for page in range(200):
    response = requests.get(url)
    data = response.json()
    my_data.extend(data['response']['results'])
    page = page + 1

In [69]:
# just made a dataframe out of created list
df = pd.DataFrame(my_data)
logging.info("make a dataframe out of the json information we have exctracted")
df

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName
0,world/2023/feb/08/canadas-justin-trudeau-greet...,article,world,World news,2023-02-08T01:13:46Z,Canada’s Justin Trudeau greets political oppon...,https://www.theguardian.com/world/2023/feb/08/...,https://content.guardianapis.com/world/2023/fe...,False,pillar/news,News
1,world/2023/mar/07/canada-pm-justin-trudeau-spe...,article,world,World news,2023-03-07T00:18:42Z,Justin Trudeau to appoint special rapporteur t...,https://www.theguardian.com/world/2023/mar/07/...,https://content.guardianapis.com/world/2023/ma...,False,pillar/news,News
2,world/2022/dec/08/sovereignty-act-passed-alber...,article,world,World news,2022-12-08T16:53:29Z,Alberta ‘sovereignty act’ sets province on col...,https://www.theguardian.com/world/2022/dec/08/...,https://content.guardianapis.com/world/2022/de...,False,pillar/news,News
3,world/2023/mar/24/trudeau-biden-asylum-border-...,article,world,World news,2023-03-24T21:41:14Z,Trudeau to announce US-Canada asylum deal afte...,https://www.theguardian.com/world/2023/mar/24/...,https://content.guardianapis.com/world/2023/ma...,False,pillar/news,News
4,tv-and-radio/2022/nov/02/drugs-impotence-and-p...,article,tv-and-radio,Television & radio,2022-11-02T17:57:23Z,"Drugs, impotence and punching Justin Trudeau: ...",https://www.theguardian.com/tv-and-radio/2022/...,https://content.guardianapis.com/tv-and-radio/...,False,pillar/arts,Arts
...,...,...,...,...,...,...,...,...,...,...,...
1995,world/2023/mar/09/justin-trudeau-china-electio...,article,world,World news,2023-03-09T16:16:12Z,Trudeau faces fury of Canadian MPs over allege...,https://www.theguardian.com/world/2023/mar/09/...,https://content.guardianapis.com/world/2023/ma...,False,pillar/news,News
1996,world/2023/feb/11/new-unidentified-object-shot...,article,world,World news,2023-02-12T07:28:59Z,New ‘unidentified object’ shot down over Canad...,https://www.theguardian.com/world/2023/feb/11/...,https://content.guardianapis.com/world/2023/fe...,False,pillar/news,News
1997,world/2022/sep/08/canada-queen-elizabeth-death...,article,world,World news,2022-09-08T19:48:01Z,"‘Canada is in mourning,’ says Justin Trudeau a...",https://www.theguardian.com/world/2022/sep/08/...,https://content.guardianapis.com/world/2022/se...,False,pillar/news,News
1998,world/2023/apr/28/canada-federal-workers-strik...,article,world,World news,2023-04-28T18:45:21Z,Federal workers tell Trudeau to step in to pay...,https://www.theguardian.com/world/2023/apr/28/...,https://content.guardianapis.com/world/2023/ap...,False,pillar/news,News


# **2. Save the result as a file in the local folder**

In [71]:
# save the dataframe to a local folder as a csv file
df.to_csv(r"C:\Users\Lion\Desktop\Test.csv",sep=",")
logging.info("save dataframe as a csv file in local folder to put it on the database")

# **3.Read the file and import it into the database using “Bulk Insert”**

*we can import the csv file in database by our hand and then truncate the table to have just columns and then do the bulk inserting by below code*

In [72]:
# Connect to SQL Server
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=OMID\SQLEXPRESS;'
                      'Database=AdventureWorks2012;'
                      'Trusted_Connection=yes;')

# Create a cursor object
cursor = conn.cursor()


# Execute the BULK INSERT command
bulk_insert_query = "BULK INSERT [AdventureWorks2012].[dbo].[irancell] FROM 'C:\\Users\\Lion\\Desktop\\Test.csv' WITH (FORMAT = 'CSV', FIRSTROW = 2)"
cursor.execute(bulk_insert_query)

# Commit the transaction
conn.commit()

# Close the cursor and connection objects
cursor.close()
conn.close()
logging.info("WE HAVE DONE IT SUCESSFULLY")

*if the quantity of table's columns was low we can create a table by query in python script and then do the bulk insert.*

In [73]:
# # Connect to SQL Server
# conn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=OMID\SQLEXPRESS;'
#                       'Database=AdventureWorks2012;'
#                       'Trusted_Connection=yes;')

# # Create a cursor object
# cursor = conn.cursor()


# # Execute the BULK INSERT command
# try:
#     create_table_query = "CREATE TABLE customers (name VARCHAR(255), address VARCHAR(255))"
#     bulk_insert_query = "BULK INSERT [AdventureWorks2012].[dbo].[irancell] FROM 'C:\\Users\\Lion\\Desktop\\Test.csv' WITH (FORMAT = 'CSV', FIRSTROW = 2)"
#     cursor.execute(create_table_query)
#     cursor.execute(bulk_insert_query)
# # if the try raise an error the except would handle it, cause of error is the table have been created before so we just do the bulk insert
# except:
#     bulk_insert_query = "BULK INSERT [AdventureWorks2012].[dbo].[irancell] FROM 'C:\\Users\\Lion\\Desktop\\Test.csv' WITH (FORMAT = 'CSV', FIRSTROW = 2)"
#     cursor.execute(bulk_insert_query)
# # Commit the transaction
# conn.commit()

# # Close the cursor and connection objects
# cursor.close()
# conn.close()
# logging.info("WE HAVE DONE IT SUCESSFULLY")

# **4. Do require a cleansing process on the data**

*convert web publication date to datetime*

In [74]:
df['webPublicationDate'] = pd.to_datetime(df['webPublicationDate'])

*check different section names*

In [75]:
df.groupby(df['sectionName']).size()

sectionName
Environment            200
Television & radio     200
World news            1600
dtype: int64

In [76]:
df.groupby(df['webPublicationDate']).size()

webPublicationDate
2022-09-08 19:48:01+00:00    200
2022-11-02 17:57:23+00:00    200
2022-12-08 12:15:00+00:00    200
2022-12-08 16:53:29+00:00    200
2023-02-08 01:13:46+00:00    200
2023-02-12 07:28:59+00:00    200
2023-03-07 00:18:42+00:00    200
2023-03-09 16:16:12+00:00    200
2023-03-24 21:41:14+00:00    200
2023-04-28 18:45:21+00:00    200
dtype: int64

In [77]:
df[df["isHosted"]==True]

,id,type,sectionId,sectionName,webPublicationDate,webTitle,webUrl,apiUrl,isHosted,pillarId,pillarName


In [78]:
df = df.drop(["isHosted","apiUrl"], axis=1)

In [88]:
df.count().notna()

id                    True
type                  True
sectionId             True
sectionName           True
webPublicationDate    True
webTitle              True
webUrl                True
pillarId              True
pillarName            True
dtype: bool

# **How to schedule python script to run every day at an exact time**

# __*first step*__

![](https://github.com/showmidelo/Irancell_Task/blob/main/schedule%20pictures/1.jpg?raw=true)

# __*Second step*__

![](https://github.com/showmidelo/Irancell_Task/blob/main/schedule%20pictures/2.jpg?raw=true)

# __*Third step*__

![](https://github.com/showmidelo/Irancell_Task/blob/main/schedule%20pictures/3.jpg?raw=true)

# __*Fourth step*__

![](https://github.com/showmidelo/Irancell_Task/blob/main/schedule%20pictures/4.jpg?raw=true)